# add language model stuff 11-8-19

In [1]:
import PIL
from PIL import ImageDraw, ImageFont, Image
def crop_ADD_image(box, img:PIL.Image):
    '''
    given a likely object; 
    crop box --> to conv-net --> lang model
    OR, if predicting, crop box --> conv-net --> retrieve prediction + lang model
    '''
    # 8-10 from s19/Model.py
    size = 224
    
    print("image shape",np.array(img).shape)
    
    img_resized = img.resize(size=(size, size))    
    cleanImg = img_resized.copy();
    
    print("image shape",np.array(cleanImg).shape)
    
    bottom = box[0] * 224
    left = box[1] * 224
    top = box[2] * 224
    right  = box[3] * 224
    # these box coordinates are not appropriate for the formatting task...
    
    print("\n*****************\ncropping?",np.array(cleanImg.crop((left,top,right,bottom))))
    
    formattedBox = (left, top, right, bottom) # coordinates need to be corrected for crop
    
    print('crop here:', formattedBox)
    
    croppedImg = cleanImg.crop(formattedBox)
    
    croppedImg = np.array(croppedImg) 
    print('cropped image shape',croppedImg.shape)
    croppedImg = croppedImg[:, :, ::-1].copy()
    
    languageFeed.append(croppedImg) 
#     return croppedImg

In [2]:
def predict(robot):
    global speak
    global prediction
    preds = []
    for i in range(len(languageFeed)):
        currentImg = languageFeed.pop()
        preds.append(predModel.predictImageWord(currentImg))
    preds = [item[0] for item in preds] #remove probability number in tuple
    prediction = collections.Counter(preds).most_common(1)[0][0]
#     speak = True
    robot.say_text(prediction).wait_for_completed()
    #return prediction

In [3]:
import pickle
import sys

In [4]:
dir_path = '/home/david/GIT/cs481-senior-design/s19/'
sys.path.append(dir_path)

import LanguageModel

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [5]:
dir_path = '/home/david/GIT/cs481-senior-design/s19/'
with open(dir_path+'language-model.pickle', 'rb') as handle:
    predModel = pickle.load(handle) #trying to load a LanguageModel type    

/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


_____________


In [6]:
dir_path = '/home/david/GIT/language-acquisition/'
sys.path.append(dir_path)

import matplotlib
# matplotlib.use("Qt5Agg") #Makes imshow work on mac
from matplotlib import pyplot as plt
from classes.mask_rcnn import MaskRCNN
#from Model import Model, draw_boxes

import queue
import time
import threading
import numpy as np
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-

In [7]:
import os

obj_det_path = "/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/tensorflow/models/research/object_detection/"

# faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12'
MODEL_NAME = obj_det_path + 'd2s_model'

path = MODEL_NAME + '/frozen_inference_graph.pb'
image_dimensions = (240, 320, 3)
model = MaskRCNN(path, image_dimensions)

PATH_TO_LABELS = os.path.join(obj_det_path + 'd2s_model', 'label_map.pbtxt')

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [8]:
import datetime
sys.path.append('/home/david/GIT/cozmo-python-sdk/')
sys.path.append('/home/david/GIT/cozmo-python-sdk/src')
sys.path.append('/home/david/GIT/cozmo-python-sdk/src/cozmo')

from src.cozmo.util import degrees, Angle, distance_mm, speed_mmps
from src import cozmo

/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [9]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def handle_image(evt, obj=None, tap_count=None,  **kwargs):
    try:
        if(imageQueue.empty()): # 
            imageQueue.put_nowait(evt.image)
    except queue.Full:
        pass

def configure_camera(robot, exposure_amount, gain_amount, color):
    robot.camera.color_image_enabled = color
    # Lerp exposure between min and max times
    min_exposure = robot.camera.config.min_exposure_time_ms
    max_exposure = robot.camera.config.max_exposure_time_ms
    exposure_time = (1-exposure_amount)*min_exposure + exposure_amount*max_exposure
    # Lerp gain
    min_gain = robot.camera.config.min_gain
    max_gain = robot.camera.config.max_gain
    actual_gain = (1-gain_amount)*min_gain + gain_amount*max_gain
    robot.camera.set_manual_exposure(exposure_time, actual_gain)

def detect_object(robot, minimum_threshold=0.10):
    '''
    run object detection on images from imageQueue
    returns: populates objectQueue with Object data. 
    '''
    print('Detect Images started')
    counter = 0
    while(True):
        if(not imageQueue.empty()): 
            try:
                img = imageQueue.get()
                image_np = load_image_into_numpy_array(img)
                output_dict = model.detect(image_np)

                indices = [list(output_dict['detection_scores']).index(o) 
                           for o in output_dict['detection_scores'] 
                           if o > minimum_threshold]
                if len(indices) > 0 and objQueue.empty():    
                    for i in range(len(indices)):
                        boxes = [output_dict['detection_boxes'][i] for i in indices]
                        b = boxes[i]
                        real_obj = obj_check(b)
                        if not real_obj[0]: 
                            continue
                        else: 
                            classes = [output_dict['detection_classes'][i] for i in indices]
                            scores = [output_dict['detection_scores'][i] for i in indices]
                            bestObj_i = i 
                            box = boxes[bestObj_i]
                            score = scores[bestObj_i]
                            label = classes[bestObj_i]

                            if objQueue.empty():
                                plt.clf() # We need to clear the plot so that we are not plotting every image each iteration. 
                                # (If we don't we will get increasing delay)
                                objQueue.put_nowait(box)
                                
                                # add image to the language feed
                                crop_ADD_image(box, img.copy())
                                
                                vis_util.visualize_boxes_and_labels_on_image_array(
                                    image_np,
                                    output_dict['detection_boxes'],
                                    output_dict['detection_classes'],
                                    output_dict['detection_scores'],
                                    category_index,
                                    use_normalized_coordinates=True,
                                    min_score_thresh=minimum_threshold,
                                    line_thickness=8,
                                    max_boxes_to_draw=3)
                                plt.imshow(image_np)
                                plt.pause(0.001) # imshow needs time to plot the image. Need this to display the image
                                
            except queue.Empty:
                pass

In [10]:
def obj_check(go_box):
    '''
    tf documentation is wrong --
    order is not: [ymin, ymax, xmin, xmax]
    order IS: [ymin, xmin, ymax, xmax]
    '''
    ymin = go_box[0]
    xmin = go_box[1]
    ymax = go_box[2]
    xmax = go_box[3]
    height = ymax - ymin
    width = xmax - xmin
    area = height * width    
    x_center = (width/2) + xmin
    y_center = (height/2) + ymin
    # the condition that determines if a bounding box contains an object
    if area > 0.08: 
#         print('check failed!')
        return (False, y_center, x_center)
    else: 
#         print('check passed')
        return (True, y_center, x_center)

In [11]:
def find_coordinates(go_box):
    real_obj = obj_check(go_box)    
    x_center = real_obj[2]
    y_center = real_obj[1]
    print('y:', y_center, 'x:', x_center)
    if not real_obj[0]: # break 'go_to' loop
        drive_dist = 1000 
        turn_angle = 1000
    else:
        # we want x_center to be 0.5, turn right -> (-) / turn left -> (+)
        if x_center > 0.56 or x_center < 0.44:
            x_diff = x_center - 0.5
            turn_angle = (x_diff * (-50)) / 2 
        else: turn_angle = 0 # break 'go_to' loop
        # we want y-center to be = 0.42, (-) -> backwards, (+) -> forwards   
        if y_center > 0.48 or y_center < 0.36:
            y_diff = y_center - 0.42
            # need to account for parallax -- things far away approach slowly; things close up approach quickly
            base = y_diff * (-10)
            drive_dist = np.power(base,4)
        else: drive_dist = 0 # break 'go_to' loop
    return drive_dist, turn_angle

def go_to_obj(robot):    
    last_drive = 1000
    last_turn = 1000
    while True:
        if not objQueue.empty():
            box = objQueue.get()
            print("go to ", box)
            drive_dist, turn_angle = find_coordinates(box)
            print('turned ', turn_angle, 'degrees', '/ drove', drive_dist,'mm')
            # these conditions break 'go_to' loop
            if drive_dist > last_drive: break
            elif turn_angle > last_turn: break
            if turn_angle == 0 and drive_dist == 0: break
            robot.turn_in_place(degrees(turn_angle),in_parallel=True)
            last_turn = turn_angle
            robot.drive_straight(distance_mm(drive_dist), speed_mmps(40), should_play_anim=False, in_parallel=True)
            last_drive = drive_dist
            robot.wait_for_all_actions_completed()  
    print('Stop going to object')
    return drive_dist, turn_angle 

In [12]:
import random
def find_object(robot):
    print('Find objects started')
    
    confidence = 0 # added for language prediction
    
    while True:
        if objQueue.empty():
#             robot.turn_in_place(degrees(360), speed=Angle(0.3), accel=Angle(0.3))
            robot.drive_wheels(l_wheel_speed=-10, r_wheel_speed=10)
        else:
            robot.stop_all_motors()
            robot.wait_for_all_actions_completed()
            # the 'go to object reorientation loop'
            predicting = False
            drive, turn = go_to_obj(robot)
            if drive == 0 and turn == 0:    
                break
            else:
                print('find object failed') 
#                 predict(robot)
                pass
            print('return to object search')
    print('Object found!')
    predict(robot)
#     predicting = True # predict in this instance

In [13]:
def cozmo_program(robot: cozmo.robot.Robot):
    print('starting pose',robot.pose)
    robot.clear_idle_animation()
    robot.stop_freeplay_behaviors()
    robot.set_idle_animation(cozmo.anim.Triggers.NeutralFace)   
    robot.set_lift_height(1.0).wait_for_completed()
    robot.set_head_angle(cozmo.robot.MIN_HEAD_ANGLE).wait_for_completed()

    exposure_amount = 0.1 # Range: [0,1]
    gain_amount = 1.0 # Range: [0,1]
    color = True
    configure_camera(robot, exposure_amount, gain_amount, color)
    
    robot.add_event_handler(cozmo.camera.EvtNewRawCameraImage, handle_image)
    print("Added event handler")
    
    minimum_threshold = 0.10
    
    threading.Thread(target=find_object, args=(robot,)).start()
    threading.Thread(target=detect_object, args=(robot,minimum_threshold,)).start() # 
    
    while True:
        time.sleep(0.1)

In [14]:
%matplotlib inline

In [15]:
import collections
imageQueue = queue.Queue(maxsize=1)
objQueue = queue.Queue(maxsize=1)
languageFeed = collections.deque(maxlen=8)

cozmo.run_program(cozmo_program, use_viewer=True)

2019-11-08 13:09:36,334 cozmo.general INFO     App connection established. sdk_version=1.4.11.dev0 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2019-11-08 13:09:36,335 cozmo.general INFO     Found robot id=1
2019-11-08 13:09:36,341 cozmo.general INFO     Connected to Android device serial=ZY223325P4
2019-11-08 13:09:36,466 cozmo.general INFO     Robot id=1 serial=024086a7 initialized OK
2019-11-08 13:09:36,576 cozmo.general INFO     Sending abort request for all actions


starting pose <Pose <Position x: 0.00 y: 0.00 z: 0.00> <Quaternion q0: 1.00 q1: 0.00 q2: 0.00 q3: -0.02 (angle_z: <Angle -0.04 radians (-2.01 degrees)>)> origin_id=7>
Added event handler
Find objects started
Detect Images started
image shape (240, 320, 3)
image shape (224, 224, 3)

*****************
cropping? <PIL.Image.Image image mode=RGB size=24x0 at 0x7F8B683E2ED0>
crop here: (0.0, 22.59876322746277, 24.122873544692993, 0.0)
cropped image shape ()
go to  [0.         0.         0.10088734 0.1076914 ]
y: 0.05044366791844368 x: 0.05384569987654686
turned  11.153857503086329 degrees / drove 186.51879111875104 mm


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-ae6683edccd8>", line 63, in detect_object
    crop_ADD_image(box, img.copy())
  File "<ipython-input-1-4722d80c58f3>", line 35, in crop_ADD_image
    croppedImg = croppedImg[:, :, ::-1].copy()
IndexError: too many indices for array

2019-11-08 13:10:16,095 cozmo.general INFO     Shutting down connection
2019-11-08 13:10:16,122 cozmo.general INFO     Android serial=ZY223325P4 disconnected.
2019-11-08 13:10:16,143 cozmo.general INFO     Exit requested by user


<Figure size 432x288 with 0 Axes>

______________________

**Good neutral animations?**

    # DroneModeIdle, 161
    # NeutralFace, 326
    # OnboardingIdle, 362
    # VC_NoFollowupCommand_NoFace, 549

In [16]:
'''
# RAMBLING AROUND + AVOIDING CLIFF CODE 

        time.sleep(0.1)
        l_turn = random.choice([0,-10,-20,30])
        l_speed = 40 + l_turn
        r_turn = random.choice([0,-10,20,-30])
        r_speed = 40 + r_turn 
#         print(r_speed, l_speed)        
        if robot.is_cliff_detected: # or not robot.is_localized
            robot.stop_all_motors()
            robot.wait_for_all_actions_completed() 
            
            #sometimes, the robot skips 'drive wheels' and goes straight into a turn
            #with disastrous results...
            
            correct = random.choice([-145,145])
            robot.drive_wheels(l_wheel_speed=-60, r_wheel_speed=-60, duration=1.75)
            robot.wait_for_all_actions_completed()
            print(correct)
            if robot.has_in_progress_actions:
                robot.drive_wheels(l_wheel_speed=-60, r_wheel_speed=-60, duration=1.75)
                robot.wait_for_all_actions_completed()
                robot.turn_in_place(degrees(correct), speed=Angle(30)).wait_for_completed()
        else:
'''

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-17b9544c6c30>", line 26, in <module>
    '''
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/IPython/core/displayhook.py", line 268, in __call__
    self.finish_displayhook()
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/ipykernel/displayhook.py", line 77, in finish_displayhook
    sys.stderr.flush()
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/ipykernel/iostream.py", line 351, in flush
    if not evt.wait(self.flush_timeout):
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/threading.py", line 552, in wait
    signaled = self._cond.wait(timeout)
  File "/home/david/anaconda3/envs/cozmo/lib/python3.7/threading.py", line 300, in wait
    gotit = waiter.acquire(True, timeout)
K

KeyboardInterrupt: 

In [ ]:
#       obj_lock.targeted and last_obj.same_object():
#         if objQueue.empty():
#         else:
#             box = objQueue.get()
#             go_to_object(robot, box)

#   obj_diff, avg_certainty = last_obj.check_again(y_center, x_center, score)
#   obj_lock.add(turn, drive)

In [ ]:
# class obj_wrapper:
#     def __init__(self, ymin, ymax, xmin, xmax):    
#         self.height = ymax - ymin
#         self.width = xmax - xmin
#         self.area = self.height * self.width
#         self.centroid = (0,0)
        
#         self.position = (-1,-1,-1)
        

In [ ]:
# import math
# class obj_certainty:
#     '''
#     an aggregate of the last several images from the Image Queue
#     '''
#     def __init__(self):
#         self.last_xcenter=0.0
#         self.last_ycenter=0.0
#         self.total_score=0.0
#         self.check_ctr=0
        
#         self.avg_score = 0.1
#         self.dist = 100
        
#     def check_again(self, y_center, x_center, score):
#         '''
#         check (Euclidean) distance between the 
#         center of this box in the frame and the previously seen box
#         '''
#         self.check_ctr+=1
#         if self.check_ctr is not 1:
#             self.dist = math.sqrt((x_center - self.last_xcenter)**2 + 
#                          (y_center - self.last_ycenter)**2)
#         else:
#             self.dist = 0
            
#         self.total_score += score
#         avg_score = self.total_score/self.check_ctr  
#         return self.dist, self.avg_score
    
#     def average_score(self, score):
#         return self.avg_score
    
#     def num_checks(self):
#         return self.check_ctr
    
#     def same_object(self):
#         if self.avg_score > 0.099 and self.dist < 400:
#             same = True # make these more specific
#         else:
#             same = False
#         return same
        
#             # different object.      
#     # Which object am I seeing the most? / Which bounding box is the most consistent?
#     # Which avg. bounding box has the most consistently high probability?

In [ ]:
# class robot_obj_lock:
    
#     def __init__(self):
#         self.moves=0
#         self.turn_angle=0
#         self.drive_dist=0
#         self.targeted=True
#         self.lost_target=False
        
#     def has_moved(self):
#         if self.turn_angle or self.drive_dist:
#             return True
#         else:
#             return False
        
#     def num_moves(self):
#         return self.moves
    
#     def add(self, turn, drive):
#         self.drive_dist+=drive
#         self.turn_angle+=turn
#         self.moves+=1
# #         if self.moves > 5:
# #             self.targeted = False
# #             self.lost_target=True
            
#         if self.turn_angle > 50 or self.turn_angle < -50:
#             self.targeted = False
#             self.lost_target=True
        
#         if self.drive_dist > 200 or self.drive_dist < -100:
#             self.targeted = False
#             self.lost_target=True

In [ ]:
# def find_obj(robot):
#      while(True):
#         if(not objQueue.empty()) and not robot.is_moving:
#     robot.set_lift_height(1.0).wait_for_completed()
#     robot.set_head_angle(cozmo.robot.MIN_HEAD_ANGLE).wait_for_completed()
    
#     found_obj = None

#     while not found_obj:
#         robot.turn_in_place(degrees(20), in_parallel=True) 
#         turn_action = None
                
#         if robot.world.visible_face_count() > 0:
#             print("Found object!")
#             face_to_follow = next(robot.world.visible_faces, None)
#             robot.wait_for_all_actions_completed()
#         if turn_action:
#             # Complete the turn action if one was in progress
#             turn_action.wait_for_completed()

#         time.sleep(.1)
    
#     print("Found object!")